# Finite Displacements

This package used finite displacement (FD) in combination with pertubation theory to compute phonons and their 
interactions. We take higher order derivatives on top of the energy derivatives that are available in DFT
engines. For example, most DFT engines computes forces using Hellmann-Feynman theorem, and in that case,
we need take 1 order of FD derivative for phonon and 2 orders of derivatives for cubic interactions.

Here we present a generic tool kit to compute finite displacements that will be used throughout the package.

## FiniteDisplacements

The `FiniteDisplacements` class computes finite displacements for a given measurement at a given order, a series of displacment amplitutes (deltas) for a given structure.



In [3]:
from principia_materia.io_interface.vasp import parse_poscar
from principia_materia.translation_group import get_structure, CrystalFTG
from principia_materia.phonon_id.finite_displacements import FiniteDisplacements

In [5]:
structure = get_structure(parse_poscar("nacl/POSCAR"), stype=CrystalFTG)
structure.orbitals = "p"

In [6]:
supa = np.identity(3) * 2
fd = FiniteDisplacements(
  structure=structure,
  supa=supa,
  fdtype="c",
  )

In [7]:
displacements = np.zeros((2, ) + fd.supercell.positions.shape)
displacements[0][0, 0] = 1
displacements[1][1, 0] = 1
fd.set_displacement_vectors(dispvecs=displacements, displabels=[1, 2])
fd.set_delta([0.01, 0.02])

In [8]:
fd.create_dft_jobs(
    directory="./test_jobs",
    dft_engine="vasp",
    jobname_prefix="testrun",
    dry_run=True,
)

['testrun_d0',
 'testrun_d1',
 'testrun_d2',
 'testrun_d3',
 'testrun_d4',
 'testrun_d5',
 'testrun_d6',
 'testrun_d7']

After DFT runs are finished, compute the finite displacements with the following codes:

```python
fd.collect_data_from_dft_jobs(
    data_type="forces",
    directory="./test_jobs",
    dft_engine="vasp",
)
fd.compute_finite_displacements()
```

## FDSeries

The `FDSeries` class offers an interface between our LID and BID analysis tools and the `FiniteDisplacements` class, making the management of a series of FD computations convenient. It also creates an easily readble directory hierarchy for these jobs. 

In [9]:
from principia_materia.phonon_id.fd_series import FDSeries

In [16]:
fdseries = FDSeries(
  structure=structure,
  supa=supa,
  order=2,
  )

In [17]:
displacements = np.zeros((2, ) + fdseries.supercell.positions.shape)
displacements[0][0, 0] = 1
displacements[1][1, 0] = 1

In [18]:
fdseries.set_measurements(measurements=displacements)
fdseries.set_deltas([0.01, 0.02, 0.03])

In [19]:
fdseries.create_dft_jobs(
    root_directory="test_jobs_series",
    dft_engine="vasp",
    jobname_prefix="the_test_job_",
    dry_run=True
    )

In order to compute the derivatives after the DFT jobs are finished, the following method can be called.
```python
fdseries.get_fd_results(
    root_directory="test_jobs_series",
    dft_engine="vasp",
    data_type="forces",
    )
```

In [21]:
help(FDSeries)

Help on class FDSeries in module principia_materia.phonon_id.fd_series:

class FDSeries(builtins.object)
 |  FDSeries(structure, supa, order, fdtype='c', tol=1e-06)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, structure, supa, order, fdtype='c', tol=1e-06)
 |      Parameters
 |      ----------
 |      order : int
 |          Order of the derivative.
 |  
 |  create_dft_jobs(self, root_directory, dft_engine, delta_format=<function format_delta_dirname at 0x7fc8339518c0>, jobname_prefix='', config_file='finite_displacements.yml', dft_config_path=None, dry_run=False)
 |  
 |  get_errortail_results(self, pick_min=3, pick_max=None, consecutive=False, penalty=<function penalty_linear_mse at 0x7fc833d7f830>, seperate_complex=True, return_pick=False, return_penalty=False)
 |  
 |  get_fd_results(self, root_directory, data_type, dft_engine, config_file='finite_displacements.yml')
 |  
 |  project_fd_results(self, basis)
 |  
 |  set_deltas(self, deltas, append=False)
 |  
 |  set_meas